# Geographic Travelling Salesman Problems
This workbook provides a framework for solving travelling salesman problems on geographic data. I wrote a Medium article about this here (_insert link_) which includes more detailed explanations.
## 1. Installation of Python libraries and Concorde
The below commands should work in bash and Mac OS. Unfortunately, Windows is not supported.

Run the following commands to create a Python virtual environment and install the required libraries:
```
virtualenv venv                                  # create virtual environment
source venv/bin/activate                         # activate it
pip install routingpy numpy folium pykml
```
Now download the Concorde executable from [this page](https://www.math.uwaterloo.ca/tsp/concorde/downloads/downloads.htm). Make it executable and place it on your path (follow [this guide](https://askubuntu.com/questions/322772/how-do-i-add-an-executable-to-my-search-path) if you don't know how to do this). This is mandatory, since the Python code below will implicitly run concorde as a command in the background. Now run the following commands to install PyConcorde:
```
git clone https://github.com/jvkersch/pyconcorde # clone git repo
cd pyconcorde                                    # change directory
git checkout pyconcorde-subprocess               # switch branch
pip install -e .                                 # install pyconcorde
```
Now get a free GraphHopper api key at [GraphHopper.com](https://www.graphhopper.com/), and either export it as an environment variable...
```
export GRAPHHOPPER_API_KEY=xyz
```
... or hard-code directly in the Python code below.
## 2. Setting up

In [217]:
import routingpy as rp
import numpy as np
import os
from concorde import Problem, run_concorde
import folium

def symmetricize(m, high_int=None):
    
    # if high_int not provided, make it equal to 10 times the max value:
    if high_int is None:
        high_int = round(10*m.max())
        
    m_bar = m.copy()
    np.fill_diagonal(m_bar, 0)
    u = np.matrix(np.ones(m.shape) * high_int)
    np.fill_diagonal(u, 0)
    m_symm_top = np.concatenate((u, np.transpose(m_bar)), axis=1)
    m_symm_bottom = np.concatenate((m_bar, u), axis=1)
    m_symm = np.concatenate((m_symm_top, m_symm_bottom), axis=0)
    
    return m_symm.astype(int) # Concorde requires integer weights

def solve_concorde(matrix):
    problem = Problem.from_matrix(matrix)
    solution = run_concorde(problem)
    print(solution.tour)
    return solution

class GeographicTSP:
    
    def __init__(self, points, profile):
        
        if isinstance(points[0], list) or isinstance(points[0], tuple):
            # List of (lon, lat) pairs
            self.points = points
            self.names = None
            
        elif isinstance(points[0], dict):
            # List of dicts of form {'name': xxx, 'lon': yyy, 'lat': zzz}
            self.points = [(p['lon'], p['lat']) for p in points]
            self.names = [p['name'] for p in points]
        else:
            raise ValueError("Invalid input format. Expected list of (lon, lat) tuples or dictionaries.")

        self.length = len(points)
        self.profile = profile
        
    def solve(self, api):
        
        matrix = api.matrix(locations=self.points, profile=self.profile)
        durations = np.matrix(matrix.durations)
        
        # test if durations is symmetric
        if np.array_equal(durations, durations.transpose()):
            # it is symmetric, do nothing
            print('distance matrix is symmetric')
            pass
        else:
            print('distance matrix is not symmetric; making it so')
            durations = symmetricize(durations)
            
        solution = solve_concorde(durations)
        
        if len(solution.tour) == self.length:
            return solution.tour
        else: 
            # check that alternate elements of solution.tour are the original points
            if max(solution.tour[0::2]) == self.length-1:
                # alternate elements (starting at index 0) are original
                self.tour = solution.tour[0::2]
                return self.tour
            else:
                # alternate elements (starting at index 1) are original
                self.tour = solution.tour[1::2]
                return self.tour
            
    def get_directions(self, api):
        
        try:
            points_ordered = [self.points[i] for i in self.tour]
            self.points_ordered = points_ordered
            if self.names is not None:
                names_ordered = [self.names[i] for i in self.tour]
                self.names_ordered = names_ordered
        except AttributeError:
            print("self.tour does not exist; ensure solve() is run first")
            
        points_ordered_with_return = points_ordered + [points_ordered[0]]
        
        directions = api.directions(locations = points_ordered_with_return, profile=self.profile)
        self.directions = directions
        return self.directions
    
    def generate_map(self):
        # Create a map centered at a specific location
        route_points = [(y, x) for (x, y) in self.points_ordered]
        centre = np.mean([x for (x, y) in route_points]), np.mean([y for (x, y) in route_points])
        
        try:
            route_line = [(y, x) for (x, y) in self.directions.geometry] # folium needs lat, long
        except AttributeError:
            print("self.directions does not exist; ensure get_directions() is run first")
        
        m = folium.Map(location=centre, zoom_start=12)

        # Create a feature group for the route line
        route_line_group = folium.FeatureGroup(name='Route Line')

        # Add the route line to the feature group
        folium.PolyLine(route_line, color='red', weight=2).add_to(route_line_group)

        # Add the feature group to the map
        route_line_group.add_to(m)

        # Create a feature group for the route points
        route_points_group = folium.FeatureGroup(name='Route Points')

        # Add the route points to the feature group
        if self.names is None:
            names = route_points
        else:
            names = self.names_ordered
        for i, (point, name) in enumerate(zip(route_points, names)):
            folium.Marker(location=point, tooltip=f'{i}: {name}').add_to(route_points_group)

        # Add the feature group to the map
        route_points_group.add_to(m)

        # Create a custom tile layer with a partially greyed out basemap
        custom_tile_layer = folium.TileLayer(
            tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
            attr='CartoDB Positron',
            name='Positron',
            overlay=True,
            control=True,
            opacity=0.7  # Adjust opacity to control the level of greying out
        )

        # Add the custom tile layer to the map
        custom_tile_layer.add_to(m)

        # Add layer control to the map
        folium.LayerControl().add_to(m)

        self.map = m
        return m

## 3. Solving TSP on a simple example
First we can solve a small TSP on a few points. Create with a list/tuple of `long, lat` pairs, and a [Graphhopper profile](https://github.com/graphhopper/graphhopper/blob/master/docs/core/profiles.md).

In [218]:
coordinates = [
                [-1.8162, 53.3651],
                [-1.8764, 53.3973],
                [-1.8757, 53.3630],
                [-1.7714, 53.3649],
                [-1.9098, 53.3578],
                [-1.9173, 53.3637],
                [-1.8826, 53.3803],
                [-1.7963, 53.3893],
                [-1.8096, 53.3492]
              ]

edale = GeographicTSP(points=coordinates, profile='hike')

In [219]:
api_key = os.environ['GRAPHHOPPER_API_KEY'] # get a free key at https://www.graphhopper.com/
api = rp.Graphhopper(api_key=api_key)

In [220]:
tour = edale.solve(api=api)

distance matrix is not symmetric; making it so
concorde /tmp/tmp6bndd7l2/problem.tsp
Host: mike-HP-EliteDesk-800-G4-WKS-TWR  Current process id: 9533
Using random seed 1687180833
Number of Nodes: 18
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 31563 (from tour)
Basic dual change required, but no candidate edges
  LP Value  1: 26033.000000  (0.00 seconds)
  LP Value  2: 31563.000000  (0.00 seconds)
New lower bound: 31563.000000
Final lower bound 31563.000000, upper bound 31563.000000
Exact lower bound: 31563.000000
DIFF: 0.000000
Final LP has 23 rows, 46 columns, 131 nonzeros
Optimal Solution: 31563.00
Number of bbnodes: 1
Total Running Time: 0.00 (seconds)
[0, 9, 3, 12, 8, 17, 2, 11, 4, 13, 5, 14, 6, 15, 1, 10, 7, 16]


ERROR: No dual change in basis finding code
Did not find a basic optimal solution
Fractional matching routine failed
No warmstart, stumbling on anyway


In [221]:
tour

[0, 3, 8, 2, 4, 5, 6, 1, 7]

The reason for the error shown above is not known and may be for technical reasons related to the large edge weights allocated between disconnected nodes. However, Concorde still managed to produce a valid solution which it claims is optimal.

The returned `tour` shows the order of the original points in the TSP solution. Note that this sequence contains alternate members of the concorde solution just above. This is because the problem was asymmetric, and `solve()` contained a step to make it into a symmetric problem with double the number of nodes, adding a ghost node for every real node. See the Medium article (link above) for an explanation on this. A set of directions can now be obtained for this solution.

In [222]:
edale.get_directions(api=api)

Direction([(-1.8162, 53.3651), (-1.816, 53.3648), (-1.81575, 53.36454), (-1.81537, 53.36425), (-1.81514, 53.36431), (-1.81482, 53.36434), (-1.81353, 53.36432), (-1.81276, 53.36435), (-1.81227, 53.36434), (-1.81192, 53.36426), (-1.81132, 53.36395), (-1.81104, 53.36386), (-1.81082, 53.36383), (-1.81009, 53.36383), (-1.80962, 53.36377), (-1.8093, 53.36375), (-1.80896, 53.36379), (-1.8081, 53.36397), (-1.8075, 53.36407), (-1.80742, 53.36388), (-1.80737, 53.36376), (-1.80716, 53.36349), (-1.8068, 53.36313), (-1.80672, 53.36293), (-1.80675, 53.36257), (-1.80671, 53.3625), (-1.80656, 53.36246), (-1.80628, 53.36248), (-1.80593, 53.36245), (-1.80585, 53.36236), (-1.80589, 53.36226), (-1.80607, 53.36213), (-1.80613, 53.36202), (-1.80598, 53.36169), (-1.80566, 53.36123), (-1.8053, 53.3609), (-1.80506, 53.36055), (-1.8047, 53.36022), (-1.80443, 53.36011), (-1.80424, 53.36006), (-1.80359, 53.35956), (-1.80356, 53.35952), (-1.80334, 53.3595), (-1.80284, 53.35912), (-1.80254, 53.35893), (-1.80238, 53

## 3. Visualization
Below is a folium map of the solution, including hover data over each point showing its place in the sequence.

In [223]:
edale.generate_map()

## 4. A larger example from Google Earth
   It is possible to plot a set of points on Google Earth and export the coordinates as a KML file, and create a `GeographicTSP` object based on the coordinates with their names. This time, the profile `'car'` is specified.

In [224]:
from pykml import parser

kml_file = os.path.join('london.kml')

with open(kml_file) as f:
  doc = parser.parse(f).getroot()

points = []

for x in doc.Document.Placemark:
    name = str(x.name)
    coords = str(x.Point.coordinates).split(',')
    lon = round(float(coords[0]), 4)
    lat = round(float(coords[1]), 4)
    points.append({'name': name, 'lon': lon, 'lat': lat})
    
points

[{'name': 'London Transport Museum', 'lon': -0.1217, 'lat': 51.5119},
 {'name': 'Leicester Square', 'lon': -0.1301, 'lat': 51.5103},
 {'name': 'Imperial War Museum', 'lon': -0.1086, 'lat': 51.4957},
 {'name': 'The Gherkin', 'lon': -0.0804, 'lat': 51.5144},
 {'name': 'Royal Albert Hall', 'lon': -0.1772, 'lat': 51.501},
 {'name': 'Soho', 'lon': -0.137, 'lat': 51.5136},
 {'name': 'Victoria & Albert Museum', 'lon': -0.1726, 'lat': 51.4969},
 {'name': 'London Zoo', 'lon': -0.1534, 'lat': 51.5353},
 {'name': 'The British Library', 'lon': -0.1277, 'lat': 51.53},
 {'name': 'US Embassy', 'lon': -0.1322, 'lat': 51.4826},
 {'name': 'O2', 'lon': 0.0034, 'lat': 51.5026},
 {'name': 'Queen Mary University', 'lon': -0.0399, 'lat': 51.5236},
 {'name': 'Emirates Stadium', 'lon': -0.1082, 'lat': 51.5549},
 {'name': 'London Eye', 'lon': -0.1195, 'lat': 51.5032},
 {'name': 'Battersea Park', 'lon': -0.1572, 'lat': 51.4817},
 {'name': 'Buckingham Palace', 'lon': -0.1409, 'lat': 51.5017},
 {'name': 'Madame Tu

In [225]:
london = GeographicTSP(points, profile='car')
london.points

[(-0.1217, 51.5119),
 (-0.1301, 51.5103),
 (-0.1086, 51.4957),
 (-0.0804, 51.5144),
 (-0.1772, 51.501),
 (-0.137, 51.5136),
 (-0.1726, 51.4969),
 (-0.1534, 51.5353),
 (-0.1277, 51.53),
 (-0.1322, 51.4826),
 (0.0034, 51.5026),
 (-0.0399, 51.5236),
 (-0.1082, 51.5549),
 (-0.1195, 51.5032),
 (-0.1572, 51.4817),
 (-0.1409, 51.5017),
 (-0.1546, 51.5229),
 (-0.1139, 51.5248),
 (-0.1163, 51.5236),
 (-0.1388, 51.5215),
 (-0.1655, 51.5074),
 (-0.0097, 51.4829),
 (-0.0238, 51.5076),
 (-0.0195, 51.5049),
 (-0.0167, 51.5385),
 (-0.0679, 51.5108),
 (-0.0819, 51.5012),
 (-0.1025, 51.528)]

In [226]:
london.names

['London Transport Museum',
 'Leicester Square',
 'Imperial War Museum',
 'The Gherkin',
 'Royal Albert Hall',
 'Soho',
 'Victoria & Albert Museum',
 'London Zoo',
 'The British Library',
 'US Embassy',
 'O2',
 'Queen Mary University',
 'Emirates Stadium',
 'London Eye',
 'Battersea Park',
 'Buckingham Palace',
 'Madame Tussauds',
 'The Postal Museum',
 'Charles Dickens Museum',
 'BT Tower',
 'Hyde Park',
 'Cutty Sark',
 'Museum of London Docklands',
 'Canary Wharf',
 'London Stadium',
 'Jack The Ripper Museum',
 'Fashion and Textile Museum',
 'City University']

In [227]:
london.solve(api)

distance matrix is not symmetric; making it so
concorde /tmp/tmp5qrzofi8/problem.tsp
Host: mike-HP-EliteDesk-800-G4-WKS-TWR  Current process id: 9537
Using random seed 1687180859
Number of Nodes: 56
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 18757 (from tour)
Basic dual change required, but no candidate edges
  LP Value  1: 17888.000000  (0.00 seconds)
  LP Value  2: 18463.333333  (0.01 seconds)
  LP Value  3: 18616.333333  (0.01 seconds)
  LP Value  4: 18656.166667  (0.02 seconds)
  LP Value  5: 18667.500000  (0.04 seconds)
  LP Value  6: 18686.500000  (0.04 seconds)


ERROR: No dual change in basis finding code
Did not find a basic optimal solution
Fractional matching routine failed
No warmstart, stumbling on anyway


  LP Value  7: 18703.148936  (0.19 seconds)
  LP Value  8: 18721.333333  (0.28 seconds)
  LP Value  9: 18732.750000  (0.34 seconds)
  LP Value 10: 18738.852941  (0.42 seconds)
  LP Value 11: 18739.250000  (0.48 seconds)
New lower bound: 18739.250000
Final lower bound 18739.250000, upper bound 18757.000000
Exact lower bound: 18739.250000
DIFF: 0.000000
Time for Total: 0.48 seconds (0.48 total in 1 calls)
Final LP has 103 rows, 123 columns, 1880 nonzeros
LOWER BOUND: 18739.250000   ACTIVE NODES: 1

Task 0: Branching on node 0
BBnode 0 split into 1 (18757.00X) 2 (18799.25X) (0.04 seconds)
Child 0 is pruned
Child 1 is pruned

Task 1: Exit
Optimal Solution: 18757.00
Number of bbnodes: 3
Total Running Time: 0.53 (seconds)  Branching Time: 0.04 (seconds)
[0, 29, 1, 33, 5, 47, 19, 36, 8, 40, 12, 35, 7, 44, 16, 48, 20, 32, 4, 34, 6, 43, 15, 42, 14, 37, 9, 30, 2, 41, 13, 54, 26, 53, 25, 50, 22, 51, 23, 49, 21, 38, 10, 52, 24, 39, 11, 31, 3, 55, 27, 45, 17, 46, 18, 28]


[0,
 1,
 5,
 19,
 8,
 12,
 7,
 16,
 20,
 4,
 6,
 15,
 14,
 9,
 2,
 13,
 26,
 25,
 22,
 23,
 21,
 10,
 24,
 11,
 3,
 27,
 17,
 18]

In [228]:
london.get_directions(api)

Direction([(-0.12131, 51.51163), (-0.12119, 51.5117), (-0.12106, 51.51164), (-0.12073, 51.51155), (-0.12065, 51.51152), (-0.12161, 51.5111), (-0.1211, 51.51064), (-0.12107, 51.51057), (-0.12333, 51.50965), (-0.12405, 51.5093), (-0.12501, 51.5088), (-0.12566, 51.50842), (-0.12673, 51.50787), (-0.12713, 51.5076), (-0.1273, 51.50746), (-0.12729, 51.50744), (-0.12731, 51.50736), (-0.12736, 51.50729), (-0.12742, 51.50724), (-0.12757, 51.50717), (-0.12777, 51.50715), (-0.1279, 51.50718), (-0.12802, 51.50728), (-0.12818, 51.50736), (-0.12826, 51.50739), (-0.12892, 51.50746), (-0.12899, 51.50751), (-0.13029, 51.50764), (-0.13035, 51.50772), (-0.13037, 51.50782), (-0.13031, 51.50795), (-0.12977, 51.50815), (-0.13006, 51.50847), (-0.13024, 51.50882), (-0.12984, 51.5089), (-0.12976, 51.50895), (-0.12996, 51.50946), (-0.1301, 51.50998), (-0.13005, 51.50998), (-0.13043, 51.50994), (-0.13103, 51.50982), (-0.13103, 51.50978), (-0.13106, 51.50976), (-0.13159, 51.50956), (-0.13227, 51.51027), (-0.13216

In [229]:
london.generate_map()